## Practical Exploration of the Delta Rule
University of Kent

CO636/836

Marek Grzes

Last modified 14/10/2020

This exploration complements Sec. 5.3 in O'reilly & Munakata (2000) that introduces the delta rule for the Summed Squared Error (SSE) and linear activation. We apply the delta rule to a tiny classification problem with two classes. Since we are interested in classification, the delta rule with a sigmoidal activation (i.e. logistic regression) would be more appropriate for this problem in general, but we start with the linear activation for didactic reasons. The implications of using sigmoidal activation in the delta rule are presented in Sec. 5.4 in the textbook, and we use this fact in the final questions that are at the end of this notebook.

### Generate Data

We generate a small dataset with two variables $x$ and $y$, and a binary class
attribute with values $-1$ and $1$. The data is generated from two normal
distributions in which the variables $x$ and $y$ are not uncorrelated.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# the number of data points per class
num_points = 20

# data points for class -1
data1 = np.random.multivariate_normal([1,1], [[0.8, 0.6],[0.6, 0.7]], num_points)
# data points for class 1
data2 = np.random.multivariate_normal([1,-1], [[0.8, 0.6],[0.6, 0.7]], num_points)

# plot both arrays
# note that arrays are indexed [row, col]
plt.scatter(data1[:,0], data1[:,1], c='green')
plt.scatter(data2[:,0], data2[:,1], c='blue')
plt.show()

Next, we add the class attribute in the third column, and we combine the two parts into one array.

In [ ]:
# we create one dataset with the class attribute
# 1) add one column with the class label
# create the class labels for both classes
c0 = - np.ones([num_points, 1])
c1 = np.ones([num_points, 1])
# append the class labels to both datasets
data1 = np.append(data1, c0, axis=1)
data2 = np.append(data2, c1, axis=1)
# 2) combine the two classes in one data array
data = np.append(data1, data2, axis=0)

# we shuffle the data by rows because we want positive and negative examples to be interleaved; the algorithm below will update the weights after processing one data example
np.random.shuffle(data)

# print the first 5 rows to see what the data looks like
print(data[:5])

We plot the final dataset that contains both classes in one array.

In [ ]:
plt.scatter(data[:,0], data[:,1], c=data[:,2] )
plt.show()

## Delta Rule

We will use the delta rule algorithm to solve our classification problem. Variables
$t_k$ and $o_k$ are used to be consistent with the textbook (O'reilly & Munakata 2000).

The code that is below uses the delta rule update equation: $\Delta w_{i,k}=\epsilon(t_k-o_k)s_i$ that is presented in Eq. (5.3) in O'reilly & Munakata (2000).

In [ ]:
# A coroutine that receives data and prints it
def printer():
    i = 0
    line=""
    while True:
        vec = yield
        if vec.size == 1:
            if i > 0:
                # print anything that is left in the buffer
                print(line)
                i = 0
                line = ""
        else:
            i = i + 1
            line = line + " " + str(np.round(vec,4))
            if i % 3 == 0:
                print(line)
                line = ""
                i = 0

printer_handler = printer()
# start the printer
printer_handler.send(None)

# we need 3 parameters: one for x, one for y, and one for the bias term
w = np.random.normal(0, 0.5, 3)
# learning rate
epsilon = 0.05
delta = 100
iteration = 0
precision=0.01

while delta > precision and iteration < 10:
    iteration = iteration + 1
    # to be consistent with the textbook, we will process one data point at a time
    for s in data:
        # compute the linear activation (predicted output)
        s_with_bias = np.copy(s)
        s_with_bias[2] = 1 # the last element is used as the bias activation
        o_k =  w.dot(s_with_bias) # see Eq. 5.6 in O'reilly & Munakata (2000)
        t_k = s[2]
        # the delta rule equation in the vector form
        delta_w = epsilon * (t_k - o_k) * s_with_bias # see Eq. 5.3
        w = w + delta_w
        # if the max change is tiny, we will stop learning in the while loop
        delta=np.max(np.absolute(delta_w))
        # print the current weights
        printer_handler.send(w)

# print the remaining values if there are any
printer_handler.send(np.zeros([1,1]))

## The Final Decision Boundary and the Threshold

Our class attribute has values -1 and 1 whereas the outputs of the linear model are between minus and plus infinity. This means that we need to introduce a threshold that will allow us to map the continuous output to the class value. A threshold of zero is a good default choice because it lies between -1 and 1. With that, all
activations that are negative will be mapped to class -1, and all activations that are positive or zero will be mapped to class +1.

Our decision boundary in 3D is $z=w[0]\times x + w[1]\times y + w[2]$. When the threshold is zero, this boundary in the 2D space defined by the xy-plane becomes $y = -\frac{w[0]}{w[1]}\times x-\frac{w[2]}{w[1]}$. This will be the red line in the figure below.

In [ ]:
# plot the final line

# construct the line using 100 points
x = np.linspace(np.min(data[:,0]),np.max(data[:,0]),100)
# this equation corresponds to our decision boundary on the xy-plane when the threshold is zero
y = -w[0]/w[1]*x-w[2]/w[1]
ymin = np.min(data[:,1])
ymax = np.max(data[:,1])

# plot the data and the line
plt.ylim(ymin-0.3, ymax+0.3)
# plot two classes separately
plt.scatter(data1[:,0], data1[:,1], c='green', label='Class -1' )
plt.scatter(data2[:,0], data2[:,1], c='blue', label='Class 1' )
plt.plot(x, y, '-r', label='boundary')
plt.legend()
plt.show()

## Error on Training Data

We will compute the number of errors on the training data.

In [ ]:
# we need to add bias activations in the last column
data_with_bias = np.copy(data)
data_with_bias[:,2]=1

# predictions are the outputs computed by the linear model
predictions = w.dot(data_with_bias.transpose())

# compute the summed squared error with respect to the linear activation; i.e., we want to see see how well the linear function approximates the values of our class attribute
SSE = np.sum(np.square(predictions-data[:,2]))

# this is the same threshold that we used to derive the red decision boundary above; with this threshold, our classification below will be consistent with the red line above
threshold = 0

# apply threshold to make discrete decisions
predictions[predictions<threshold] = -1
predictions[predictions>=threshold] = 1
# print(predictions)
# print(data[:,2])

# diff will be zero when the classification is correct
diff = predictions - data[:,2]
# indices of data items with incorrect predictions
erroneous = diff.nonzero()[0]
# count the number of incorrect predictions
accuracy = np.size(erroneous)

print("SSE on training data: " + str(SSE) )
print("Number of errors on training data: " + str(accuracy) )

## Questions

You are not required to answer all the questions that are below. I would encourage you to attempt the first five. The last two questions require a certain amount of coding in Python, and they can be skipped by those who are not interested in further exploration.

1. Read this notebook and analyse all sections trying to understand them and trying to see the link with the textbook and the lectures.

2. The red line in Sec. _"The Final Decision Boundary and the Threshold"_ is the decision boundary for our classification problem, and it corresponds to the threshold of zero. How would this decision boundary change if
the threshold was smaller or greater than zero? Note which data points have the class -1 and which +1; this can be seen in the legend in the figure. You may want to do subsequent questions first if this question is challenging for you initially.

2. Add computation of the Summed Squared Error (SSE) in the "while" loop of the learning algorithm and print its value. Note that we are interested in the SSE between the true class value and the linear output of our neural network because this is what the delta rule optimises.

3. In the current implementation, the final decision boundary (the red line above)
 is plotted after the learning process has finished. Your task is to plot the
 data and the current boundary in the "while" or "for" loop of the learning algorithm. The goal is to investigate how the decision boundary is changing during learning, and to contrast those changes with the changes in the SSE. You basically want to see the decision boundary after every application of the delta rule equation.

4. If you run the simulation many times, you will notice that the SSE may be non-zero even if the number of classification errors on the training data is zero. Explain how this is possible.

5. The decision boundary that is learned has the form: $z=w[0]\times x+w[1]\times y+w[2]$ where $z$ is the last column in our training data, and it represents the class attribute.  Similarly, $x$ and $y$ are the first and the second columns in the data array respectively. This means that strictly speaking the decision boundary is a plane located in a 3-dimensional xyz-space. Explain how this plane is placed with respect to the flat scatterplot of the data examples in their xy-plane (i.e. the graph with the red line). After trying to estimate the location of the separating hyperplane in 3D, you could construct a 3-dimensional version of the this plot to see where exactly this plane is located. An example 3D scatterplot in mathplotib is here https://matplotlib.org/3.1.1/gallery/mplot3d/scatter3d.html

6. Add sigmoidal activation to this implementation. Note that assuming that we are switching the Cross-Entropy error (CE) instead of the Summed Squared Error (SSE), the update equation for the delta rule remains unchanged. The only difference is the activation function that is sigmoidal and takes the original linear activation as input. This is well explained in Sec. 5.4 in O'reilly & Munakata (2000). Since the range of the sigmoid function is the interval $[0,1]$, it would make sense to replace the class -1 with 0 in our data.

## References

O'reilly, Randall C., and Yuko Munakata. Computational explorations in cognitive neuroscience: Understanding the mind by simulating the brain. MIT press, 2000.


<small>This document was written in Pycharm Professional and compiled using Python 3.8 with standard packages available in Ubuntu 20.04.</small>
